# IMDb

## Import libraries and data

In [ ]:
import pandas as pd
from scipy import stats
import pingouin as pg
import numpy as np
import statistics as stat
from operator import itemgetter
import scikit_posthocs as sp
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
data_basics = pd.read_csv ("title_basics.tsv", sep='\t')
data_ratings = pd.read_csv ("title_ratings.tsv", sep='\t')

## Clean data

In [ ]:
#not all elements in basics have corresponding ratings
# filter rows based on list values
mask = data_basics['tconst'].isin(data_ratings.tconst)
data_basics = data_basics[mask]


In [ ]:
#select only the movies with ratings
mask = data_basics['titleType'] == 'movie'
movies = data_basics[mask]

#select only the ratings of movies with ratings
mask = data_ratings['tconst'].isin(movies.tconst)
movie_ratings = data_ratings[mask]

In [ ]:
#remove the ones without start date
mask = movies['startYear'] != "\\N"
movies = movies[mask]

#remove them from ratings too
mask = data_ratings['tconst'].isin(movies.tconst)
movie_ratings = movie_ratings[mask]

In [ ]:
first_year = 2000
last_year = 2020

print("The data spans between " + str(np.min(movies['startYear'])) + " and " +
      str(np.max(pd.to_numeric(movies['startYear']))) + ".")
print("We are interested in the data between " + str(first_year) + " and " +
      str(last_year) + ".")

In [ ]:
#remove the ones without start date
mask_min = movies['startYear'].apply(pd.to_numeric) >= first_year
movies = movies[mask_min]
mask_max = movies['startYear'].apply(pd.to_numeric) <= last_year
movies = movies[mask_max]


#remove them from ratings too
mask = movie_ratings['tconst'].isin(movies.tconst)
movie_ratings = movie_ratings[mask]

print("Selected data between " + str(np.min(movies['startYear'])) + " and " +
      str(np.max(pd.to_numeric(movies['startYear']))) + ".")

In [ ]:
min_votes = 1000
mask = movie_ratings['numVotes'].apply(pd.to_numeric) >= min_votes
movie_ratings = movie_ratings[mask]

mask = movies['tconst'].isin(movie_ratings.tconst)
movies = movies[mask]

In [ ]:
#remove adult movies
mask = movies['isAdult'].apply(pd.to_numeric) == 0
movies = movies[mask]

mask = movie_ratings['tconst'].isin(movies.tconst)
movie_ratings = movie_ratings[mask]

## Analyse the data to select the test used

In [ ]:
#plot average rating results for all movies
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.hist(movie_ratings['averageRating'], color='b')

ax.set_xlabel('Average rating')
ax.set_ylabel('Number of ocurences')
ax.set_title('Average rating histogram')
ax.grid()

In [ ]:
stat_list = {
        "mean" : (stat.mean(movie_ratings['averageRating'])),
        "mode" : ((stat.mode(movie_ratings['averageRating']))),
        "median" : (stat.median(movie_ratings['averageRating'])),
        "std" : (stat.stdev(movie_ratings['averageRating'])),
        "var" : (stat.stdev(movie_ratings['averageRating']))**2}

print(stat_list)

In [ ]:
#Select genres of interest
genres = ['Comedy',
          'Drama',
          'Fantasy',
          'Horror',
          'Romance',
          'Action',
          'Thriller',
          'Sci-Fi',
          'Documentary']

# 0 drop multi-genre, 1 keep multi genre without repetition (recommended)
multi_genre = 1

match multi_genre:
    case 0:
        movies_split = {gen: movies[movies['genres'] == gen] for gen in genres}
        movie_ratings_split = {gen:
            movie_ratings[movie_ratings['tconst'].isin(movies_split[gen]['tconst'])] 
            for gen in genres}
        print([str(gen)+ ': ' + str(len(movies_split[gen])) for gen in genres])

    case 1:
        movies_split = {gen:
            movies[movies['genres'].apply(lambda x: x.split(',')[0]) == gen]
            for gen in genres}
        movie_ratings_split = {gen:
            movie_ratings[movie_ratings['tconst'].isin(movies_split[gen]['tconst'])]
            for gen in genres}
        print([str(gen)+ ': ' + str(len(movies_split[gen])) for gen in genres])

In [ ]:
# plot average rating results for each genre
fig, ax = plt.subplots(int(np.ceil(len(genres) / 3)), 3, figsize=(10,7))
ax = ax.ravel()
fig.suptitle('Average rating for different genres', fontsize=15)
for axes, i in zip(ax, range(0, len(genres))):
    axes.hist(movie_ratings_split[genres[i]]['averageRating'], color='b')
    axes.legend([genres[i]], loc="upper left", fontsize=7)
    axes.grid()
fig.text(0.5, 0.04, 'Average rating', ha='center')
fig.text(0.04, 0.5, 'Number of ocurences', va='center', rotation='vertical')


In [ ]:
#Organize data to plot in a single graph
ratings_genre = [movie_ratings_split[genre]['averageRating']
                 for genre in genres]

#plot average ratings results in a box
fig, ax = plt.subplots(figsize=(10, 7))

fig.suptitle('Average rating for different genres', fontsize=15)
ax.boxplot(ratings_genre)
ax.grid()

ax.set_xticklabels(genres, rotation=45, ha='right', fontsize=10)
fig.text(0.5, 0.04, 'Genres', ha='center')
fig.text(0.04, 0.5, 'Average rating', va='center', rotation='vertical')
plt.tight_layout()

In [ ]:
outliers_info = {}
outliers = {} 
movie_ratings_split_filter = {}
# calculate percentage of outliers
for genre in genres:
    data = movie_ratings_split[genre]['averageRating']
    q1 = np.percentile(data, 25) #first quartile
    q3 = np.percentile(data, 75) # third quartlie
    iqr = q3 - q1 #interquartile range

    #Tukey, J. W. (1977). Exploratory Data Analysis.
    #Outlier limit defined by matplotlib
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers[genre] = data[(data < lower_bound) | (data > upper_bound)]

    #percentage of the samples that are considered "outliers"
    count = len(outliers[genre])
    percentage = count / len(data) * 100
    outliers_info[genre] = f"{count} outliers ({percentage:.2f}%)"

    #filter outliers
    mask = (data >= lower_bound) & (data <= upper_bound)
    movie_ratings_split_filter[genre] = pd.DataFrame(data[mask])

# Print outlier info per genre
for genre, info in outliers_info.items():
    print(f"{genre}: {info}")

In [ ]:
#evaluate outliers

#check for wrong type
[print(outlier) for genre in genres for outlier in outliers[genre]
 if not isinstance(outlier, float)]
#check non-positive values
[print(outlier) for genre in genres for outlier in outliers[genre]
 if outlier <= 0.0]
#check values bigger than 10
[print(outlier) for genre in genres for outlier in outliers[genre]
 if outlier > 10.0]

In [ ]:
#plot relative error results without outliers
fig, ax = plt.subplots(int(np.ceil(len(genres)/3)), 3, figsize=(10,7))
ax = ax.ravel()
fig.suptitle('Average rating for different genres', fontsize=15)
for axes, i in zip(ax, range(0, len(genres))):
    axes.hist(movie_ratings_split_filter[genres[i]]['averageRating'], color='b')
    axes.legend([genres[i]], loc="upper left", fontsize=7)
    axes.grid()
fig.text(0.5, 0.04, 'Average rating', ha='center')
fig.text(0.04, 0.5, 'Number of ocurences', va='center', rotation='vertical')

In [ ]:
parameters = {}
movie_ratings_db = {}
# 1: wants to leave the outliers in # 0: wants to take the outliers out
outliers_in = 1

match outliers_in:
    case 0:
        movie_ratings_db = movie_ratings_split_filter
    case 1:
        movie_ratings_db = movie_ratings_split

# Quick sample parameters check
for genre in genres:
    parameters[genre] = {
        "mean" : (stat.mean(movie_ratings_db[genre]['averageRating'])),
        "mode" : ((stat.mode(movie_ratings_db[genre]['averageRating']))),
        "median" : (stat.median(movie_ratings_db[genre]['averageRating'])),
        "std" : (stat.stdev(movie_ratings_db[genre]['averageRating'])),
        "var" : (stat.stdev(movie_ratings_db[genre]['averageRating']))**2}

vars = list(map(itemgetter('var'), parameters.values()))

print(parameters)

print(max(vars)/min(vars))

In [ ]:
parameters = {}
movie_ratings_db = {}

# 1: wants to leave the outliers in # 0: wants to take the outliers out
outliers_in = 0

match outliers_in:
    case 0:
        movie_ratings_db = movie_ratings_split_filter
    case 1:
        movie_ratings_db = movie_ratings_split

for genre in genres:
    stat, p = stats.shapiro(movie_ratings_db[genre]['averageRating'])
    print(genre, "p =", p)
    values = movie_ratings_split[genre]['averageRating']
    print(genre,"skew =", stats.skew(values), "kurtosis =", stats.kurtosis(values))

Possible courses of action:
-Resample with the same sample size + regular anova: pros: balanced design + simple cons: heteroscedaticity, maybe small sample size could impact normality
-Regular anova (accept ratio as okay): pros: quick and easy, higher sample sizes (?) cons:  heteroscedaticity, unbalanced design
-Welch ANOVA or Brown-Forsythe (with post-hoc analysis): pros: more robust to unbalanced sample sizes and heteroscedasticity cons: higher story points

## Run ANOVA (and Ad Hoc tests)

In [ ]:
parameters = {}
movie_ratings_db = {}

# 1: wants to leave the outliers in # 0: wants to take the outliers out (recommended)
outliers_in = 0

match outliers_in:
    case 0:
        movie_ratings_db = movie_ratings_split_filter
    case 1:
        movie_ratings_db = movie_ratings_split

In [ ]:
samples = np.array([movie_ratings_db[genre]['averageRating'].values
                    for genre in genres], dtype=object)

# Since the sample ratio is higher than 2, regular ANOVA is not suitable
# Applied Linear Statistical Models by Kutner et all
# Welch’s ANOVA test with different sample sizes
f_welch, p_welch = stats.f_oneway(*samples, axis=0, equal_var=False)
print(f_welch)
print(p_welch)

In [ ]:
movie_ratings_flat= pd.DataFrame(
    [(genre, rating)
     for genre in genres
     for rating in movie_ratings_db[genre]['averageRating']],
    columns=['genre', 'averageRating']
)

In [ ]:
# Post Hoc analysis
# Which groups differ?
# Games-Howell test

pg.pairwise_gameshowell(data=movie_ratings_flat, dv='averageRating', between='genre')

In [ ]:
df = movie_ratings_flat.set_index('genre')
args = df.groupby(df.index)['averageRating'].apply(list)
# Scipy Games Howell
res = stats.tukey_hsd(*args, equal_var= False)
conf = res.confidence_interval()
print("Genre A | Genre B | low interval | high interval | p-value")
for ((i, j), l) in np.ndenumerate(conf.low):
    # filter out self comparisons and repetitions
    if j > i:
        h = conf.high[i,j]
        p = res.pvalue[i,j]
        print(f"({genres[i]} - {genres[j]}) {l:>6.3f} {h:>6.3f} {p:>6.3f}")

## Median Analysis

In [ ]:
# Mood’s Median Test
stats.median_test(*args, ties='ignore')

In [ ]:
stats.kruskal(*args)

In [ ]:
sp.posthoc_dunn(movie_ratings_flat,
                val_col='averageRating',
                group_col='genre',
                p_adjust='bonferroni')